In [121]:
import pandas as pd
import numpy as np
from joblib import load
import joblib
import os

In [130]:
import pandas as pd
import numpy as np
import heejin
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, LabelEncoder, RobustScaler, \
    PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR, SVC
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error, \
    classification_report

dataDF = pd.read_csv('../data/Hamburger.csv', encoding='latin1')
dataDF.info()

dataDF['restaurant'].replace('macdonald', 'McDonalds', inplace=True)
dataDF['restaurant'].replace('Mcdonald', 'McDonalds', inplace=True)
dataDF['restaurant'].replace('Mcdonalds', 'McDonalds', inplace=True)
dataDF['restaurant'].replace('burgerking', 'Burger King', inplace=True)

# 중복값 제거 - 2개 
dataDF.duplicated().sum()
dataDF.drop_duplicates(inplace=True)

# 결측치 제거 - protein - 94개
dataDF.isna().sum()
dataDF.dropna(subset='protein', inplace=True)

# 전처리한 데이터 파일로 
dataDF.to_csv('../data/Hambuger_2.csv')
# 타겟 : calories
# 피쳐 : sodium, sugar, total_fat, portein
targetSR = dataDF['calories']
featureDF = dataDF[dataDF.columns[3:]]
targetSR.shape, featureDF.shape
# 학습용 데이터셋, 검증용 데이터셋 분리
xtrain, xtest, ytrain, ytest = train_test_split(featureDF, targetSR, test_size=0.2, random_state=48)
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)


feature=dataDF[["sugar","sodium","total_fat","protein"]]
target=dataDF["calories"]

x_train,x_test, y_train,y_test=train_test_split(feature, target, test_size=0.3, random_state=42)

new_data = [[10,1345,13,37]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   restaurant  1068 non-null   object 
 1   item        1068 non-null   object 
 2   calories    1068 non-null   float64
 3   sodium      1068 non-null   float64
 4   sugar       1068 non-null   float64
 5   total_fat   1068 non-null   float64
 6   protein     974 non-null    float64
dtypes: float64(5), object(2)
memory usage: 58.5+ KB


In [131]:
knn = KNeighborsRegressor(n_neighbors=2, weights='distance', p=1)
knn.fit(xtrain, ytrain)

li = LinearRegression(fit_intercept=False, copy_X=False, n_jobs=100)
li.fit(xtrain, ytrain)

ridge = load('../model/Ridge.pkl')
ridge.fit(x_train, y_train)

ridgeCV = load('../model/RidgeCV.pkl')
ridgeCV.fit(x_train, y_train)

bayridge = load('../model/BayesianRidge.pkl')
bayridge.fit(x_train, y_train)

kridge = load('../model/KernelRidge.pkl')
kridge.fit(x_train, y_train)

lasso = load('../model/Lasso.pkl')
lasso.fit(x_train, y_train)

lassoCV = load('../model/LassoCV.pkl')
lassoCV.fit(x_train, y_train)

lassoLars = load('../model/LassoLars.pkl')
lassoLars.fit(x_train, y_train)

dt = load('../model/decision_tree_total_02.pkl')
dt.fit(xtrain, ytrain)

boost = load('../model/boosting_total.pkl')
boost.fit(xtrain, ytrain)

rf = load('../model/RandomForest.pkl')
rf.fit(xtrain, ytrain)

vt = load('../model/Voting.pkl')
vt.fit(xtrain, ytrain)

Convergence after  3  iterations


VotingRegressor(estimators=[('LI',
                             LinearRegression(copy_X=False, fit_intercept=False,
                                              n_jobs=100)),
                            ('KNN',
                             KNeighborsRegressor(n_neighbors=2, p=1,
                                                 weights='distance')),
                            ('Ridge',
                             Ridge(alpha=0.9, max_iter=1000, random_state=28,
                                   solver='lsqr')),
                            ('Lasso',
                             Lasso(alpha=1.5, max_iter=10000, positive=True,
                                   selection='random', tol=0.01)),
                            ('DT',
                             DecisionTreeRegressor(crit...
                                                   min_impurity_decrease=3,
                                                   min_samples_leaf=3,
                                                   min_samples_split=4)),
                            ('Boost',
                             AdaBoostRegressor(estimator=DecisionTreeRegressor(criterion='friedman_mse',
                                                                               max_depth=7,
                                                                               max_features=3,
                                                                               min_impurity_decrease=3,
                                                                               min_samples_leaf=3,
                                                                               min_samples_split=4))),
                            ('RandomForest',
                             RandomForestRegressor(max_depth=10,
                                                   max_features='sqrt',
                                                   min_samples_split=5,
                                                   n_estimators=50,
                                                   random_state=18))])

In [132]:
predictDict = {'예측 모델':[],'예측 값':[],'실제 값':[],'MAE':[],'RMSE':[]}

In [133]:
# 피쳐 : sodium, sugar, total_fat, portein
new_data = [[1345,10,13,37]]
scaler = StandardScaler()
new_data = scaler.fit_transform(new_data)
new_data2 = [[10,1345,13,37]]
true_y = [759]

In [134]:
predictDict['예측 모델'].append('KNeighborsRegressor')
predictDict['예측 값'].append(knn.predict(new_data)[0].round(3))
predictDict['실제 값'].append(true_y[0])
predictDict['MAE'].append(mean_absolute_error(true_y,knn.predict(new_data)))
predictDict['RMSE'].append(mean_squared_error(true_y,knn.predict(new_data), squared=False))

predictDict['예측 모델'].append('LinearRegression')
predictDict['예측 값'].append(li.predict(new_data)[0].round(3))
predictDict['실제 값'].append(true_y[0])
predictDict['MAE'].append(mean_absolute_error(true_y,li.predict(new_data)))
predictDict['RMSE'].append(mean_squared_error(true_y,li.predict(new_data), squared=False))

predictDict['예측 모델'].append('Ridge')
predictDict['예측 값'].append(ridge.predict(new_data2)[0].round(3))
predictDict['실제 값'].append(true_y[0])
predictDict['MAE'].append(mean_absolute_error(true_y,ridge.predict(new_data2)))
predictDict['RMSE'].append(mean_squared_error(true_y,ridge.predict(new_data2), squared=False))

predictDict['예측 모델'].append('RidgeCV')
predictDict['예측 값'].append(ridgeCV.predict(new_data2)[0].round(3))
predictDict['실제 값'].append(true_y[0])
predictDict['MAE'].append(mean_absolute_error(true_y,ridgeCV.predict(new_data2)))
predictDict['RMSE'].append(mean_squared_error(true_y,ridgeCV.predict(new_data2), squared=False))

predictDict['예측 모델'].append('BayesianRidge')
predictDict['예측 값'].append(bayridge.predict(new_data2)[0].round(3))
predictDict['실제 값'].append(true_y[0])
predictDict['MAE'].append(mean_absolute_error(true_y,bayridge.predict(new_data2)))
predictDict['RMSE'].append(mean_squared_error(true_y,bayridge.predict(new_data2), squared=False))

predictDict['예측 모델'].append('Lasso')
predictDict['예측 값'].append(lasso.predict(new_data2)[0].round(3))
predictDict['실제 값'].append(true_y[0])
predictDict['MAE'].append(mean_absolute_error(true_y,lasso.predict(new_data2)))
predictDict['RMSE'].append(mean_squared_error(true_y,lasso.predict(new_data2), squared=False))

predictDict['예측 모델'].append('LassoCV')
predictDict['예측 값'].append(lassoCV.predict(new_data2)[0].round(3))
predictDict['실제 값'].append(true_y[0])
predictDict['MAE'].append(mean_absolute_error(true_y,lassoCV.predict(new_data2)))
predictDict['RMSE'].append(mean_squared_error(true_y,lassoCV.predict(new_data2), squared=False))

predictDict['예측 모델'].append('LassoLars')
predictDict['예측 값'].append(lassoLars.predict(new_data2)[0].round(3))
predictDict['실제 값'].append(true_y[0])
predictDict['MAE'].append(mean_absolute_error(true_y,lassoLars.predict(new_data2)))
predictDict['RMSE'].append(mean_squared_error(true_y,lassoLars.predict(new_data2), squared=False))

predictDict['예측 모델'].append('DecisionTree')
predictDict['예측 값'].append(dt.predict(new_data)[0].round(3))
predictDict['실제 값'].append(true_y[0])
predictDict['MAE'].append(mean_absolute_error(true_y,dt.predict(new_data)))
predictDict['RMSE'].append(mean_squared_error(true_y,dt.predict(new_data), squared=False))

predictDict['예측 모델'].append('Boosting')
predictDict['예측 값'].append(boost.predict(new_data)[0].round(3))
predictDict['실제 값'].append(true_y[0])
predictDict['MAE'].append(mean_absolute_error(true_y,boost.predict(new_data)))
predictDict['RMSE'].append(mean_squared_error(true_y,boost.predict(new_data), squared=False))

predictDict['예측 모델'].append('RandomForest')
predictDict['예측 값'].append(rf.predict(new_data)[0].round(3))
predictDict['실제 값'].append(true_y[0])
predictDict['MAE'].append(mean_absolute_error(true_y,rf.predict(new_data)))
predictDict['RMSE'].append(mean_squared_error(true_y,rf.predict(new_data), squared=False))

predictDict['예측 모델'].append('Voting')
predictDict['예측 값'].append(vt.predict(new_data)[0].round(3))
predictDict['실제 값'].append(true_y[0])
predictDict['MAE'].append(mean_absolute_error(true_y,vt.predict(new_data)))
predictDict['RMSE'].append(mean_squared_error(true_y,vt.predict(new_data), squared=False))

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RidgeCV was fitted with

In [135]:
predictDF = pd.DataFrame(predictDict)
predictDF-

,예측 모델,예측 값,실제 값,MAE,RMSE
0,KNeighborsRegressor,447.292,759,311.707854,311.707854
1,LinearRegression,0.000,759,759.000000,759.000000
2,Ridge,499.914,759,259.085587,259.085587
3,RidgeCV,504.449,759,254.550842,254.550842
4,BayesianRidge,500.740,759,258.260229,258.260229
5,Lasso,494.420,759,264.579784,264.579784
6,LassoCV,496.170,759,262.829565,262.829565
7,LassoLars,495.930,759,263.070039,263.070039
8,DecisionTree,537.333,759,221.666667,221.666667
9,Boosting,578.500,759,180.500000,180.500000
